In [33]:
import torch
import gzip
import csv
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
import time
import math
from torch.utils.data import Dataset, DataLoader

In [34]:
HIDDEN_SIZE = 100
BATCH_SIZE = 256
N_LAYER = 2
N_EPOCHS = 10
N_CHARS = 128
USE_GPU = False

In [35]:
def create_tensor(tensor):
    if USE_GPU:
        device = torch.device("cuda:0")
        tensor = tensor.to(device)
    return tensor
def name2list(name):
    arr = [ord(c) for c in name]
    return arr, len(arr)
def make_tensors(names, countries):
    sequences_and_lengths = [name2list(name) for name in names]
    name_sequences = [sl[0] for sl in sequences_and_lengths]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequences_and_lengths])
    countries = countries.long()
    # make tensor of name, BatchSize x SeqLen
    seq_tensor = torch.zeros(len(name_sequences), seq_lengths.max()).long()
    for idx, (seq, seq_len) in enumerate(zip(name_sequences, seq_lengths), 0):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    # sort by length to use pack_padded_sequence
    seq_lengths, perm_idx = seq_lengths.sort(dim=0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    countries = countries[perm_idx]
    return  create_tensor(seq_tensor),\
            create_tensor(seq_lengths),\
            create_tensor(countries)
            #BatchSize x SeqLen,BatchSize,BatchSize

In [12]:

#name2list('hello'),torch.tensor([104, 101, 108, 108, 111]).sort(dim=0, descending=True),name2list('hello')[0][torch.tensor([1,2])]
torch.tensor([104, 101, 108, 108, 111])[torch.tensor([1,0])]

tensor([101, 104])

In [36]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = 2 if bidirectional else 1
        self.embedding = torch.nn.Embedding(input_size, hidden_size)
        self.gru = torch.nn.GRU(hidden_size, hidden_size, n_layers,
        bidirectional=bidirectional)
        self.fc = torch.nn.Linear(hidden_size * self.n_directions, output_size)
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
        batch_size, self.hidden_size)
        return create_tensor(hidden)
    def forward(self, input, seq_lengths):
        # input shape : B x S -> S x B
        input = input.t()
        batch_size = input.size(1)
        hidden = self._init_hidden(batch_size)
        embedding = self.embedding(input)
        # pack them up
        gru_input = nn.utils.rnn.pack_padded_sequence(embedding, seq_lengths)
        output, hidden = self.gru(gru_input, hidden)
        if self.n_directions == 2:
            hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim=1)
        else:
            hidden_cat = hidden[-1]
        fc_output = self.fc(hidden_cat)
        return fc_output

In [37]:
class NameDataset(Dataset):
    def __init__(self, is_train_set=True):
        filename = 'data/names_train.csv.gz' if is_train_set else 'data/names_test.csv.gz'
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f)
            rows = list(reader)
        self.names = [row[0] for row in rows]
        self.len = len(self.names)
        self.countries = [row[1] for row in rows]
        self.country_list = list(sorted(set(self.countries)))
        self.country_dict = self.getCountryDict()
        self.country_num = len(self.country_list)
    def __getitem__(self, index):
        return self.names[index], self.country_dict[self.countries[index]]
    def __len__(self):
        return self.len
    def getCountryDict(self):
        country_dict = dict()
        for idx, country_name in enumerate(self.country_list, 0):
            country_dict[country_name] = idx
        return country_dict
    def idx2country(self, index): 
            return self.country_list[index]
    def getCountriesNum(self):
        return self.country_num

In [38]:
testset = NameDataset(is_train_set=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
trainset = NameDataset(is_train_set=True)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
N_COUNTRY = trainset.getCountriesNum()

In [39]:
trainset.getCountryDict()

{'Arabic': 0,
 'Chinese': 1,
 'Czech': 2,
 'Dutch': 3,
 'English': 4,
 'French': 5,
 'German': 6,
 'Greek': 7,
 'Irish': 8,
 'Italian': 9,
 'Japanese': 10,
 'Korean': 11,
 'Polish': 12,
 'Portuguese': 13,
 'Russian': 14,
 'Scottish': 15,
 'Spanish': 16,
 'Vietnamese': 17}

In [40]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
def trainModel():
    total_loss = 0
    for i, (names, countries) in enumerate(trainloader, 1):
        inputs, seq_lengths, target = make_tensors(names, countries)
        output = classifier(inputs, seq_lengths)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if i % 10 == 0:
            print(f'[{time_since(start)}] Epoch {epoch} ', end='')
            print(f'[{i * len(inputs)}/{len(trainset)}] ', end='')
            print(f'loss={total_loss / (i * len(inputs))}')
    return total_loss
def testModel():
    correct = 0
    total = len(testset)
    print("evaluating trained model ...")
    with torch.no_grad():
        for i, (names, countries) in enumerate(testloader, 1):
            inputs, seq_lengths, target = make_tensors(names, countries)
            output = classifier(inputs, seq_lengths)
            pred = output.max(dim=1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        percent = '%.2f' % (100 * correct / total)
        print(f'Test set: Accuracy {correct}/{total} {percent}%')
    return correct / total


classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRY, N_LAYER)
if USE_GPU:
    device = torch.device("cuda:0")
    classifier.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
start = time.time()
print("Training for %d epochs..." % N_EPOCHS)
acc_list = []
for epoch in range(1, N_EPOCHS + 1):
# Train cycle
    trainModel()
    acc = testModel()
    acc_list.append(acc)
    

Training for 10 epochs...
 3s] Epoch 1 [2560/13374] loss=0.008864981774240731
 5s] Epoch 1 [5120/13374] loss=0.007563631841912865
 7s] Epoch 1 [7680/13374] loss=0.006881272746250033
 9s] Epoch 1 [10240/13374] loss=0.0064178886939771475
 11s] Epoch 1 [12800/13374] loss=0.0060393330920487645
evaluating trained model ...
Test set: Accuracy 4514/6700 67.37%
 14s] Epoch 2 [2560/13374] loss=0.00406025790143758
 16s] Epoch 2 [5120/13374] loss=0.004083089565392583
 18s] Epoch 2 [7680/13374] loss=0.004014090821146965
 20s] Epoch 2 [10240/13374] loss=0.003938831842970103
 22s] Epoch 2 [12800/13374] loss=0.003855418264865875
evaluating trained model ...
Test set: Accuracy 4947/6700 73.84%
 25s] Epoch 3 [2560/13374] loss=0.003352350159548223
 27s] Epoch 3 [5120/13374] loss=0.0032870883238501846
 29s] Epoch 3 [7680/13374] loss=0.0031968329567462206
 31s] Epoch 3 [10240/13374] loss=0.003120268328348175
 33s] Epoch 3 [12800/13374] loss=0.0030941906291991473
evaluating trained model ...
Test set: Accu

In [23]:
node = [1, 2, 3]
# file = open('node.txt', 'a')
# #mid = str(node)
# # mid = str(node).replace('[', '').replace(']', '')
# # # 删除单引号并用字符空格代替逗号
# # mid = mid.replace("'", '').replace(',', '') + '\n'
# file.write(node)
# file.close()

with open('node.txt', 'a') as f:
    f.write(str(node).replace('[', '').replace(']', '').replace(',', ''))

In [24]:
import numpy as np

node_pair1 = np.loadtxt('./node.txt', dtype=bytes).astype(int)
node_pair = list(node_pair1)
print(node_pair)


[1, 2, 3]
